In [1]:
import pickle
import sys
import objects
import string
import re
import enchant
from bs4 import BeautifulSoup
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
import os

In [2]:
data_dir = '../Data/nd'

filenames = os.listdir(data_dir)
filepaths = [os.path.join(data_dir, filename) for filename in filenames]
num_files = len(filenames)

count_dicts = []

In [3]:
with open('../Data/count_dicts_nd.py', 'rb') as wc_file:
    unpickler = pickle.Unpickler(wc_file)
    count_dicts = unpickler.load()

In [28]:
print('unpickler loaded')
w = enchant.Dict("en_US")

for i, count_dict in enumerate(count_dicts):
    for key in count_dict.keys():
        # remove proper nouns
        # remove numbers/other symbols
        if not w.check(key) or not key.isalpha():
            del count_dict[key]

unpickler loaded


KeyboardInterrupt: 

In [5]:
v = DictVectorizer(sparse = True)
X = v.fit_transform(count_dicts)
fnames = v.get_feature_names()
with open('../Data/feature_names.pkl', 'wb') as fn_file:
    pickle.dump(fnames, fn_file)

In [7]:
fnames

[u'a',
 u'ab',
 u'abaft',
 u'abandon',
 u'abandoned',
 u'abandoning',
 u'abandonment',
 u'abandons',
 u'abashed',
 u'abate',
 u'abated',
 u'abatement',
 u'abatements',
 u'abates',
 u'abating',
 u'abattoir',
 u'abbey',
 u'abbeys',
 u'abbot',
 u'abbreviate',
 u'abbreviated',
 u'abbreviates',
 u'abbreviating',
 u'abbreviation',
 u'abbreviations',
 u'abdicate',
 u'abdicated',
 u'abdicates',
 u'abdicating',
 u'abdication',
 u'abdomen',
 u'abdominal',
 u'abduct',
 u'abducted',
 u'abducting',
 u'abduction',
 u'abductions',
 u'abductor',
 u'abducts',
 u'abed',
 u'aberrant',
 u'aberration',
 u'aberrational',
 u'aberrations',
 u'abet',
 u'abets',
 u'abetted',
 u'abetting',
 u'abettor',
 u'abettors',
 u'abeyance',
 u'abhor',
 u'abhorrent',
 u'abhors',
 u'abide',
 u'abides',
 u'abiding',
 u'abilities',
 u'ability',
 u'abject',
 u'abjures',
 u'ablaze',
 u'able',
 u'abler',
 u'ables',
 u'ablest',
 u'ably',
 u'abnormal',
 u'abnormalities',
 u'abnormality',
 u'abnormally',
 u'aboard',
 u'abode',
 u'ab

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_v = TfidfTransformer()
TX = tfidf_v.fit_transform(X)

In [11]:
lda = LatentDirichletAllocation(n_components=10, learning_method = 'online')
ft = lda.fit_transform(TX)

In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [32]:
lda.components_[0, 2:100]

array([0.10000044, 0.1       , 0.1       , 0.10000001, 0.1       ,
       0.10000001, 0.10000065, 0.1       , 0.10000001, 0.1       ,
       0.10000002, 0.10000001, 0.10000001, 0.10000058, 0.10000001,
       0.10000037, 0.10000071, 0.1000006 , 0.10000001, 0.10000068,
       0.10000047, 0.10000001, 0.10000002, 0.10000002, 0.10000007,
       0.10000029, 0.1000002 , 0.10000017, 0.10000001, 0.10000001,
       0.10000004, 0.10000003, 0.10000008, 0.10000001, 0.10000006,
       0.10000054, 0.1000002 , 0.10000054, 0.1000001 , 0.10000002,
       0.10000043, 0.10000033, 0.10000002, 0.10000006, 0.10000001,
       0.10000003, 0.10000051, 0.10000079, 0.10000001, 0.10000016,
       0.10000002, 0.10000007, 0.1       , 0.10000004, 0.10000001,
       0.1       , 0.1       , 0.10000067, 0.10000049, 0.10000054,
       0.1       , 0.10000041, 0.1000007 , 0.10000025, 0.10000001,
       0.10000001, 0.10000001, 0.10000001, 0.10000001, 0.10000005,
       0.10000001, 0.10000071, 0.10000001, 0.1       , 0.10000

In [36]:
TX[2]

<1x36124 sparse matrix of type '<type 'numpy.float64'>'
	with 1254 stored elements in Compressed Sparse Row format>

In [18]:
print_top_words(lda, fnames, 100)

Topic #0: checkpoint checkpoints insignias turnoffs hijackers miscast matted choreographed hedger nonrandom immigrants scripted festivities chock rallies fluorescent interrupts belittle biologist stimulants beguiled freshening deduces taunt eradicating fatalities wheelings exquisite multipurpose restarting wipers subjectiveness protocols intrusiveness daunting mutilation uniforms medley rally roadblocks unsettling patrols traffickers suppresses politically greet logged arsenal pant empirical multiply curving purges singularly invasions sprays finite roadblock adjustable smuggling combating apprehending manpower cones roving flagged fatigued researchers destinations engendered unconstrained frighten fresheners unbridled spit condemns exalt maximize wrappers motorists uniformed accumulates visually setup aliens blouse motoring flares societal greeting wheeling slaughter misread randomly alleviating bounce masking vitality invests cautions
Topic #1: domesticating shiv sewn defiling defaci

In [19]:
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [24]:
print_top_words(lda, fnames, 100)

Topic #0: the of and to in a that is for by property or city not be on as was from it land which use v this at its public are an water with court we oil building commission section lease no gas construction line north were damages value has have plaintiffs such any would there all their evidence trial highway been owner other well company county area co also road inc service they cost if used had owners part electric township case may project right into district within defendants between feet will west railroad only river improvement upon located lot than
Topic #1: the of to and in a that is be or by for state as it this not such which statute shall section v an any are county law board with may from court under was upon has act its no on north at public school all have laws case tax we provisions chapter legislature other states within jurisdiction been if action legislative power constitution but general district said one must us so provided statutes does made only same city authorit

In [26]:
lda.fit(TX)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [27]:
print_top_words(lda, fnames, 100)

Topic #0: domesticating shiv defiling sewn trample noose defacing embroil vibe vibes junking mildew seedless squirrel gargoyle nonadjacent lugs signalization spaghetti inartistically springtime winnow whitey consort fieldwork patriarch idiomatic incognitos obliging mimeograph farina malignancies cancers aromatic cohort schoolyard apologetic unpremeditated blazed reeducate feuds postorder gelled ovens appending disestablishment underfunding toss micron grainer cholera simulating recusant participatory leaven cruises affiliating precut notching outing acceptably woeful persnickety confute fiddling retrench tornadoes rebuttals slogans seamstress muscats fairways sayer mauled codas muscat overripe raisins sunburned subsuming humanized minicomputers pedantically overcautious doubtfulness reinterpretation newcomers blossomed ermine indistinctly sequestrate comers intermarry rumbling percolate quickest sleeted sleeting yardarm horrors
Topic #1: thrasher colognes glues seasonings cleansers sce

In [ ]:
'''
with open('../Data/tfidf_matrix.pkl', 'rb') as wc_file:
    unpickler = pickle.Unpickler(wc_file)
    X = unpickler.load()

with open('../Data/feature_names.pkl', 'rb') as feature_names_file:
    unpickler = pickle.Unpickler(feature_names_file)
    fn = unpickler.load()
    
with open('../Data/frobenius.pkl', 'rb') as feature_names_file:
    unpickler = pickle.Unpickler(frobenius)
    fn = unpickler.load()

lda = LatentDirichletAllocation(n_components=5, learning_method = 'online')
ft = lda.fit_transform(X)
'''

In [ ]:
'''
...more dead code and pickle files past here
'''